PWC-Net-large model training (with multisteps learning rate schedule)
=========================================================

In this notebook, we:
- Use a PWC-Net-large model (with dense and residual connections), 6 level pyramid, uspample level 2 by 4 as the final flow prediction
- Train the model on a mix of the `FlyingChairs` and `FlyingThings3DHalfRes` dataset using a S<sub>long</sub> schedule described in [[2018a]](#2018a)
- The S<sub>long</sub> schedule is borrowed from [[2016a]](#2016a) and looks as follows:

![](img/lr_multisteps.png)

Below, look for `TODO` references and customize this notebook based on your own machine setup.

## Reference

- [2018a]<a name="2018a"></a> Sun et al. 2018. PWC-Net: CNNs for Optical Flow Using Pyramid, Warping, and Cost Volume. [[arXiv]](https://arxiv.org/abs/1709.02371) [[web]](http://research.nvidia.com/publication/2018-02_PWC-Net%3A-CNNs-for) [[PyTorch (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/PyTorch) [[Caffe (Official)]](https://github.com/NVlabs/PWC-Net/tree/master/Caffe)
- [2016a]<a name="2016a"></a> Ilg et al. 2016. FlowNet 2.0: Evolution of Optical Flow Estimation with Deep Networks. [[arXiv]](https://arxiv.org/abs/1612.01925) [[PyTorch (Official)]](https://github.com/NVIDIA/flownet2-pytorch) [[TensorFlow]](https://github.com/sampepose/flownet2-tf)

In [1]:
"""
pwcnet_train.ipynb

PWC-Net model training.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)

Tensorboard:
    [win] tensorboard --logdir=E:\\repos\\tf-optflow\\tfoptflow\\pwcnet-lg-6-2-multisteps-chairsthingsmix
    [ubu] tensorboard --logdir=/media/EDrive/repos/tf-optflow/tfoptflow/pwcnet-lg-6-2-multisteps-chairsthingsmix
"""
from __future__ import absolute_import, division, print_function
import sys
from copy import deepcopy

from dataset_base import _DEFAULT_DS_TRAIN_OPTIONS
from dataset_flyingchairs import FlyingChairsDataset
from dataset_flyingthings3d import FlyingThings3DHalfResDataset
from dataset_mixer import MixedDataset
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TRAIN_OPTIONS

## TODO: Set this first!

In [2]:
# TODO: You MUST set dataset_root to the correct path on your machine!
if sys.platform.startswith("win"):
    _DATASET_ROOT = 'E:/datasets/'
else:
    _DATASET_ROOT = '/media/EDrive/datasets/'
_FLYINGCHAIRS_ROOT = _DATASET_ROOT + 'FlyingChairs_release'
_FLYINGTHINGS3DHALFRES_ROOT = _DATASET_ROOT + 'FlyingThings3D_HalfRes'
    
# TODO: You MUST adjust the settings below based on the number of GPU(s) used for training
# Set controller device and devices
# A one-gpu setup would be something like controller='/device:GPU:0' and gpu_devices=['/device:GPU:0']
# Here, we use a dual-GPU setup, as shown below
gpu_devices = ['/device:GPU:0', '/device:GPU:1']
controller = '/device:CPU:0'

# TODO: You MUST adjust this setting below based on the amount of memory on your GPU(s)
# Batch size
batch_size = 8

# Pre-train on `FlyingChairs+FlyingThings3DHalfRes` mix

## Load the dataset

In [3]:
# TODO: You MUST set the batch size based on the capabilities of your GPU(s) 
#  Load train dataset
ds_opts = deepcopy(_DEFAULT_DS_TRAIN_OPTIONS)
ds_opts['in_memory'] = False                          # Too many samples to keep in memory at once, so don't preload them
ds_opts['aug_type'] = 'heavy'                         # Apply all supported augmentations
ds_opts['batch_size'] = batch_size * len(gpu_devices) # Use a multiple of 8; here, 16 for dual-GPU mode (Titan X & 1080 Ti)
ds_opts['crop_preproc'] = (256, 448)                  # Crop to a smaller input size
ds1 = FlyingChairsDataset(mode='train_with_val', ds_root=_FLYINGCHAIRS_ROOT, options=ds_opts)
ds_opts['type'] = 'into_future'
ds2 = FlyingThings3DHalfResDataset(mode='train_with_val', ds_root=_FLYINGTHINGS3DHALFRES_ROOT, options=ds_opts)
ds = MixedDataset(mode='train_with_val', datasets=[ds1, ds2], options=ds_opts)

In [4]:
# Display dataset configuration
ds.print_config()


Dataset Configuration:
  verbose              False
  in_memory            False
  crop_preproc         (256, 448)
  scale_preproc        None
  input_channels       3
  tb_test_imgs         False
  random_seed          1969
  val_split            0.03
  aug_type             heavy
  aug_labels           True
  fliplr               0.5
  flipud               0.5
  translate            (0.5, 0.05)
  scale                (0.5, 0.05)
  batch_size           16
  type                 into_future
  mode                 train_with_val
  train size           41282
  val size             1230


## Configure the training

In [5]:
# Start from the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TRAIN_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_dir'] = './pwcnet-lg-6-2-multisteps-chairsthingsmix/'
nn_opts['batch_size'] = ds_opts['batch_size']
nn_opts['x_shape'] = [2, ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 3]
nn_opts['y_shape'] = [ds_opts['crop_preproc'][0], ds_opts['crop_preproc'][1], 2]
nn_opts['use_tf_data'] = True # Use tf.data reader
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# Use the PWC-Net-large model in quarter-resolution mode
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

In [6]:
# Set the learning rate schedule. This schedule is for a single GPU using a batch size of 8.
# Below,we adjust the schedule to the size of the batch and the number of GPUs.
nn_opts['lr_policy'] = 'multisteps'
nn_opts['lr_boundaries'] = [400000, 600000, 800000, 1000000, 1200000]
nn_opts['lr_values'] = [0.0001, 5e-05, 2.5e-05, 1.25e-05, 6.25e-06, 3.125e-06]
nn_opts['max_steps'] = 1200000

# Below, we adjust the schedule to the size of the batch and our number of GPUs (2).
nn_opts['max_steps'] = int(nn_opts['max_steps'] * 8 / ds_opts['batch_size'])
nn_opts['lr_boundaries'] = [int(boundary * 8 / ds_opts['batch_size']) for boundary in nn_opts['lr_boundaries']]

In [7]:
# Instantiate the model and display the model configuration
nn = ModelPWCNet(mode='train_with_val', options=nn_opts, dataset=ds)
nn.print_config()

Building model towers...
  Building tower_0...
Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
  ...tower_0 built.
  Building tower_1...
  ...tower_1 built.
... model towers built.
Initializing model from previous checkpoint ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-37000 to resume training...

INFO:tensorflow:Restoring parameters from ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-37000
... model initialized

Model Configuration:
  verbose                True
  ckpt_dir               ./pwcnet-lg-6-2-multisteps-chairsthingsmix/
  max_to_keep            10
  x_dtype                <dtype: 'float32'>
  x_shape                [2, 256, 448, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [256, 448, 2]
  train_mode             train
  display_step           100
  snapshot_step          1000
  val_step               1000
  val_batch_size         -1
  tb_val_imgs            pyramid
  tb_test_imgs           None
  gp

## Train the model

In [ ]:
# Train the model
nn.train()

Resume training from step 37001...
2018-09-12 10:57:53 Iter 37100 [Train]: loss=69.17, epe=4.84, lr=0.000100, samples/sec=17.7, sec/step=1.811, eta=11 days, 19:10:29
2018-09-12 10:59:35 Iter 37200 [Train]: loss=71.01, epe=4.95, lr=0.000100, samples/sec=34.7, sec/step=0.922, eta=6 days, 0:11:35
2018-09-12 11:01:16 Iter 37300 [Train]: loss=71.67, epe=5.00, lr=0.000100, samples/sec=34.7, sec/step=0.922, eta=6 days, 0:09:31
2018-09-12 11:02:59 Iter 37400 [Train]: loss=66.80, epe=4.61, lr=0.000100, samples/sec=34.4, sec/step=0.930, eta=6 days, 1:17:35
2018-09-12 11:04:40 Iter 37500 [Train]: loss=69.68, epe=4.85, lr=0.000100, samples/sec=34.8, sec/step=0.918, eta=5 days, 23:29:12
2018-09-12 11:06:21 Iter 37600 [Train]: loss=71.76, epe=5.01, lr=0.000100, samples/sec=34.7, sec/step=0.921, eta=5 days, 23:53:22
2018-09-12 11:08:03 Iter 37700 [Train]: loss=69.31, epe=4.82, lr=0.000100, samples/sec=34.6, sec/step=0.924, eta=6 days, 0:23:50
2018-09-12 11:09:45 Iter 37800 [Train]: loss=67.14, epe=4.

2018-09-12 12:28:18 Iter 42200 [Train]: loss=65.38, epe=4.53, lr=0.000100, samples/sec=33.8, sec/step=0.946, eta=6 days, 2:36:06
2018-09-12 12:30:02 Iter 42300 [Train]: loss=68.15, epe=4.74, lr=0.000100, samples/sec=33.8, sec/step=0.947, eta=6 days, 2:39:02
2018-09-12 12:31:45 Iter 42400 [Train]: loss=67.80, epe=4.69, lr=0.000100, samples/sec=33.9, sec/step=0.945, eta=6 days, 2:21:58
2018-09-12 12:33:29 Iter 42500 [Train]: loss=68.60, epe=4.77, lr=0.000100, samples/sec=33.8, sec/step=0.948, eta=6 days, 2:44:08
2018-09-12 12:35:13 Iter 42600 [Train]: loss=64.94, epe=4.48, lr=0.000100, samples/sec=33.8, sec/step=0.946, eta=6 days, 2:31:28
2018-09-12 12:36:57 Iter 42700 [Train]: loss=66.21, epe=4.58, lr=0.000100, samples/sec=33.8, sec/step=0.947, eta=6 days, 2:38:19
2018-09-12 12:38:41 Iter 42800 [Train]: loss=63.68, epe=4.38, lr=0.000100, samples/sec=33.8, sec/step=0.948, eta=6 days, 2:39:58
2018-09-12 12:40:25 Iter 42900 [Train]: loss=69.97, epe=4.86, lr=0.000100, samples/sec=33.7, sec/

2018-09-12 14:01:44 Iter 47400 [Train]: loss=64.41, epe=4.43, lr=0.000100, samples/sec=33.0, sec/step=0.969, eta=6 days, 4:47:35
2018-09-12 14:03:30 Iter 47500 [Train]: loss=66.61, epe=4.61, lr=0.000100, samples/sec=33.2, sec/step=0.965, eta=6 days, 4:07:10
2018-09-12 14:05:15 Iter 47600 [Train]: loss=62.47, epe=4.25, lr=0.000100, samples/sec=33.1, sec/step=0.968, eta=6 days, 4:33:51
2018-09-12 14:07:01 Iter 47700 [Train]: loss=59.27, epe=4.03, lr=0.000100, samples/sec=33.1, sec/step=0.967, eta=6 days, 4:23:40
2018-09-12 14:08:47 Iter 47800 [Train]: loss=66.58, epe=4.60, lr=0.000100, samples/sec=33.0, sec/step=0.968, eta=6 days, 4:32:44
2018-09-12 14:10:34 Iter 47900 [Train]: loss=67.18, epe=4.65, lr=0.000100, samples/sec=33.0, sec/step=0.970, eta=6 days, 4:47:58
2018-09-12 14:12:20 Iter 48000 [Train]: loss=64.87, epe=4.45, lr=0.000100, samples/sec=33.0, sec/step=0.969, eta=6 days, 4:38:35
2018-09-12 14:12:36 Iter 48000 [Val]: loss=69.17, epe=4.81
Saving model...
INFO:tensorflow:./pwcn

2018-09-12 15:37:04 Iter 52600 [Train]: loss=65.35, epe=4.44, lr=0.000100, samples/sec=32.2, sec/step=0.994, eta=6 days, 7:05:05
2018-09-12 15:38:54 Iter 52700 [Train]: loss=67.78, epe=4.66, lr=0.000100, samples/sec=31.7, sec/step=1.008, eta=6 days, 9:16:10
2018-09-12 15:40:43 Iter 52800 [Train]: loss=60.45, epe=4.11, lr=0.000100, samples/sec=32.1, sec/step=0.996, eta=6 days, 7:24:41
2018-09-12 15:42:32 Iter 52900 [Train]: loss=60.23, epe=4.10, lr=0.000100, samples/sec=32.2, sec/step=0.994, eta=6 days, 7:00:22
2018-09-12 15:44:20 Iter 53000 [Train]: loss=61.27, epe=4.19, lr=0.000100, samples/sec=32.3, sec/step=0.992, eta=6 days, 6:43:44
2018-09-12 15:44:37 Iter 53000 [Val]: loss=64.82, epe=4.50
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-53000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-53000
2018-09-12 15:46:42 Iter 53100 [Train]: loss=58.29, epe=3.96, lr=

2018-09-12 17:15:27 Iter 57800 [Train]: loss=58.31, epe=3.95, lr=0.000100, samples/sec=31.2, sec/step=1.024, eta=6 days, 10:15:11
2018-09-12 17:17:18 Iter 57900 [Train]: loss=61.62, epe=4.19, lr=0.000100, samples/sec=31.3, sec/step=1.024, eta=6 days, 10:07:25
2018-09-12 17:19:10 Iter 58000 [Train]: loss=59.45, epe=4.05, lr=0.000100, samples/sec=31.2, sec/step=1.025, eta=6 days, 10:22:13
2018-09-12 17:19:28 Iter 58000 [Val]: loss=61.55, epe=4.23
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-58000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-58000
2018-09-12 17:21:36 Iter 58100 [Train]: loss=59.76, epe=4.06, lr=0.000100, samples/sec=31.3, sec/step=1.021, eta=6 days, 9:40:03
2018-09-12 17:23:28 Iter 58200 [Train]: loss=59.41, epe=4.03, lr=0.000100, samples/sec=31.2, sec/step=1.024, eta=6 days, 10:11:05
2018-09-12 17:25:19 Iter 58300 [Train]: loss=59.28, epe=4.03,

...

2018-09-12 22:23:42 Iter 73300 [Train]: loss=57.74, epe=3.86, lr=0.000100, samples/sec=29.2, sec/step=1.097, eta=6 days, 16:31:56
2018-09-12 22:25:42 Iter 73400 [Train]: loss=57.54, epe=3.86, lr=0.000100, samples/sec=29.0, sec/step=1.103, eta=6 days, 17:18:03
2018-09-12 22:27:41 Iter 73500 [Train]: loss=56.91, epe=3.82, lr=0.000100, samples/sec=29.1, sec/step=1.099, eta=6 days, 16:40:57
2018-09-12 22:29:41 Iter 73600 [Train]: loss=57.73, epe=3.87, lr=0.000100, samples/sec=29.0, sec/step=1.102, eta=6 days, 17:08:48
2018-09-12 22:31:41 Iter 73700 [Train]: loss=58.68, epe=3.93, lr=0.000100, samples/sec=29.2, sec/step=1.096, eta=6 days, 16:13:47
2018-09-12 22:33:40 Iter 73800 [Train]: loss=54.52, epe=3.62, lr=0.000100, samples/sec=29.2, sec/step=1.094, eta=6 days, 15:57:08
2018-09-12 22:35:40 Iter 73900 [Train]: loss=61.31, epe=4.10, lr=0.000100, samples/sec=29.1, sec/step=1.100, eta=6 days, 16:42:34
2018-09-12 22:37:39 Iter 74000 [Train]: loss=57.75, epe=3.87, lr=0.000100, samples/sec=29.

2018-09-13 00:11:22 Iter 78500 [Train]: loss=54.08, epe=3.59, lr=0.000100, samples/sec=28.5, sec/step=1.121, eta=6 days, 18:22:26
2018-09-13 00:13:24 Iter 78600 [Train]: loss=55.33, epe=3.71, lr=0.000100, samples/sec=28.7, sec/step=1.117, eta=6 days, 17:45:59
2018-09-13 00:15:26 Iter 78700 [Train]: loss=52.73, epe=3.51, lr=0.000100, samples/sec=28.6, sec/step=1.118, eta=6 days, 17:57:46
2018-09-13 00:17:27 Iter 78800 [Train]: loss=56.32, epe=3.77, lr=0.000100, samples/sec=28.6, sec/step=1.118, eta=6 days, 17:47:39
2018-09-13 00:19:29 Iter 78900 [Train]: loss=57.08, epe=3.82, lr=0.000100, samples/sec=28.8, sec/step=1.112, eta=6 days, 16:54:58
2018-09-13 00:21:30 Iter 79000 [Train]: loss=56.25, epe=3.77, lr=0.000100, samples/sec=28.6, sec/step=1.118, eta=6 days, 17:46:42
2018-09-13 00:21:51 Iter 79000 [Val]: loss=69.79, epe=4.78
Saving model...
... model wasn't saved -- its score (4.78) doesn't outperform other checkpoints
2018-09-13 00:24:03 Iter 79100 [Train]: loss=59.14, epe=3.96, lr=

2018-09-13 02:02:40 Iter 83800 [Train]: loss=57.76, epe=3.86, lr=0.000100, samples/sec=28.0, sec/step=1.142, eta=6 days, 19:45:56
2018-09-13 02:04:45 Iter 83900 [Train]: loss=54.53, epe=3.63, lr=0.000100, samples/sec=27.9, sec/step=1.148, eta=6 days, 20:36:27
2018-09-13 02:06:48 Iter 84000 [Train]: loss=53.28, epe=3.56, lr=0.000100, samples/sec=28.2, sec/step=1.136, eta=6 days, 18:46:41
2018-09-13 02:07:09 Iter 84000 [Val]: loss=58.13, epe=3.91
Saving model...
INFO:tensorflow:./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-84000 is not in all_model_checkpoint_paths. Manually adding it.
... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-84000
2018-09-13 02:09:29 Iter 84100 [Train]: loss=56.21, epe=3.75, lr=0.000100, samples/sec=28.2, sec/step=1.134, eta=6 days, 18:34:10
2018-09-13 02:11:34 Iter 84200 [Train]: loss=54.35, epe=3.60, lr=0.000100, samples/sec=27.9, sec/step=1.147, eta=6 days, 20:16:52
2018-09-13 02:13:38 Iter 84300 [Train]: loss=62.35, epe=4.24

... model saved in ./pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-89000
2018-09-13 03:56:06 Iter 89100 [Train]: loss=55.47, epe=3.68, lr=0.000100, samples/sec=27.6, sec/step=1.159, eta=6 days, 20:30:26
2018-09-13 03:58:13 Iter 89200 [Train]: loss=52.13, epe=3.44, lr=0.000100, samples/sec=27.4, sec/step=1.169, eta=6 days, 21:52:04
2018-09-13 04:00:20 Iter 89300 [Train]: loss=51.83, epe=3.42, lr=0.000100, samples/sec=27.2, sec/step=1.174, eta=6 days, 22:35:44
2018-09-13 04:02:27 Iter 89400 [Train]: loss=57.04, epe=3.76, lr=0.000100, samples/sec=27.5, sec/step=1.162, eta=6 days, 20:51:01
2018-09-13 04:04:32 Iter 89500 [Train]: loss=60.76, epe=4.05, lr=0.000100, samples/sec=27.7, sec/step=1.155, eta=6 days, 19:48:03
2018-09-13 04:06:39 Iter 89600 [Train]: loss=55.71, epe=3.67, lr=0.000100, samples/sec=27.5, sec/step=1.162, eta=6 days, 20:43:39
2018-09-13 04:08:46 Iter 89700 [Train]: loss=54.14, epe=3.57, lr=0.000100, samples/sec=27.4, sec/step=1.167, eta=6 days, 21:23:33
2018-09-13

2018-09-13 05:51:16 Iter 94400 [Train]: loss=50.89, epe=nan, lr=0.000100, samples/sec=27.0, sec/step=1.187, eta=6 days, 22:43:44
2018-09-13 05:53:25 Iter 94500 [Train]: loss=54.43, epe=3.61, lr=0.000100, samples/sec=26.9, sec/step=1.190, eta=6 days, 23:09:57
2018-09-13 05:55:34 Iter 94600 [Train]: loss=50.68, epe=3.32, lr=0.000100, samples/sec=27.0, sec/step=1.185, eta=6 days, 22:23:00
2018-09-13 05:57:43 Iter 94700 [Train]: loss=52.30, epe=3.42, lr=0.000100, samples/sec=27.0, sec/step=1.186, eta=6 days, 22:29:44
2018-09-13 05:59:52 Iter 94800 [Train]: loss=58.84, epe=3.89, lr=0.000100, samples/sec=27.0, sec/step=1.184, eta=6 days, 22:08:13
2018-09-13 06:02:01 Iter 94900 [Train]: loss=52.47, epe=3.46, lr=0.000100, samples/sec=27.0, sec/step=1.184, eta=6 days, 22:03:35
2018-09-13 06:04:09 Iter 95000 [Train]: loss=55.51, epe=3.70, lr=0.000100, samples/sec=27.2, sec/step=1.178, eta=6 days, 21:12:50
2018-09-13 06:04:32 Iter 95000 [Val]: loss=61.63, epe=4.11
Saving model...
INFO:tensorflow:

## Training log

TBD: Here are the training curves for the run above:

![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/loss.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/epe.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/lr.png)

TBD: Here are the predictions issued by the model for a few validation samples:

![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val1.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val2.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val3.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val4.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val5.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val6.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val7.png)
![](img/pwcnet-lg-6-2-multisteps-chairsthingsmix/val8.png)